In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from math import *
import pandas as pd
import numpy as np
from random import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer
from sklearn.model_selection import train_test_split

In [2]:
np.set_printoptions(precision=6, suppress=True)
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [3]:
def evaluate_loss(X_train, y_train, y_pred, y_test, model):
    y_pred = y_pred
    y_pred = np.array(y_pred).reshape((1,21,))
    last_data = X_train[-1]
    pred_1 = model.predict(y_pred,verbose=0)
    b = np.sqrt(np.sum(np.square(y_pred-last_data)))
    a = (np.square(y_train[-1]-pred_1))
    return pred_1

In [5]:
df = pd.read_excel('aug_nine_var.xlsx')

X = df.iloc[:,1:22]
y = df.iloc[:,22].to_numpy()

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

_,X_test,_,y_test=train_test_split(X, y, test_size=0.05, random_state=42,shuffle=False)
X_train,_,y_train,_=train_test_split(X, y, test_size=0.05, random_state=42,shuffle=True)

model = tf.keras.models.load_model('dnn.h5')

In [8]:
epochs = 1000
latent_dim = 21 
model.trainable = False
noise_shape = tf.keras.Input(shape=(latent_dim,))
my_model = tf.keras.Sequential([
    tf.keras.Input((latent_dim,)),
    tf.keras.layers.Reshape((latent_dim,)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(256,activation='selu'),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(1024,activation='relu'),
    tf.keras.layers.Dense(21,activation='relu'),
    tf.keras.layers.Activation('relu')
    ])

my_model.compile()

In [12]:
generator = tf.keras.Model(noise_shape, my_model(noise_shape))

noise_shape = tf.keras.Input(shape=(latent_dim,))
noise_shape2 = tf.keras.Input(shape=(latent_dim,))

c = tf.keras.Model(noise_shape2,model(noise_shape2))

gen_thing_to_real_model = tf.keras.Model(noise_shape, c(my_model(noise_shape)))
g_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

generator.compile(loss=evaluate_loss, optimizer=g_optimizer)
gen_thing_to_real_model.compile(loss = 'mse', optimizer = g_optimizer)

In [13]:
def evaluate_loss(y_pred, y_test):
    y_pred = y_pred
    y_pred = np.array(y_pred).reshape((1,21,))
    last_data = X_train[-1]
    pred_1 = model.predict(y_pred,verbose=0)
    # pred_2 = model.predict(y_test,verbose=0)
    b = np.sqrt(np.sum(np.square(y_pred-last_data)))
    a = (np.square(y_train[-1]-pred_1))
    return a+(np.abs(b*10-a))

In [ ]:
def train_step(real_model,generator,epochs,model):
    df = pd.read_excel('dataframe.xlsx')
    batch_size = 1
    y_train = list(y_train)
    x = []
    y = np.array([])
    result_ = np.array([])
    to_excel_list = []
    average = sum(y_train) / len(y_train)
    for j in range(epochs):
        noise = tf.random.normal([1,21])
        real_model.train_on_batch(noise,np.zeros((1,21)))
        y_pred = generator.predict(noise,verbose=0)[0]
        s = evaluate_loss(y_pred,_)
        average = model.predict(y_pred.reshape((1,21)),verbose=0)
        print("loss_value: %.4f average: %.4f try: %d" % (s,average,j))
        to_excel_list = generator.predict(noise,verbose=0)[0]
        to_excel_list = np.array(to_excel_list).reshape((1,21))
        c = to_excel_list
        to_excel_list = scaler.inverse_transform(to_excel_list)
        to_excel_list = np.append(to_excel_list,model.predict(c,verbose=0))
        result_ = np.append(result_,to_excel_list).reshape((-1,22))
        df = pd.DataFrame(result_)    
        df.to_excel("basic_formula.xlsx")